In [1]:
from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import MultiGrid


class MoneyAgent(Agent):
    """ An agent with fixed initial wealth."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 1

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def give_money(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        if len(cellmates) > 1:
            other_agent = self.random.choice(cellmates)
            other_agent.wealth += 1
            self.wealth -= 1

    def step(self):
        self.move()
        if self.wealth > 0:
            self.give_money()


class MoneyModel(Model):
    """A model with some number of agents."""
    def __init__(self, N, width, height):
        self.num_agents = N
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.agent = 0
        # Create agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self)
            self.agent = a
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))

    def step(self):
        self.schedule.step()


In [2]:
model = MoneyModel(1, 10, 10)

for i in range(100):
    model.step()

lst = model.grid.get_neighbors((0, 0), moore=True, radius=20)
print(f'surrounding visitors: {lst}')

agent = model.agent
model.grid.remove_agent(agent)
lst = model.grid.get_neighbors((0,0), moore=True, radius=20)
print(f'surrounding visitors: {lst}')

print('done')

surrounding visitors: [<__main__.MoneyAgent object at 0x7f98b98ce190>]
surrounding visitors: []
done
